# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

,City_ID,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Date
0,0,Jamestown,AU,-33.21,138.60,94,18,76.15,1.79,1572426056
1,1,Qeshm,IR,26.96,56.27,20,46,93.20,9.17,1572426056
2,2,Tual,ID,-5.67,132.75,13,75,81.73,7.85,1572426056
3,3,Port Elizabeth,US,39.31,-74.98,90,93,63.00,3.36,1572426057
4,4,Vanavara,RU,60.35,102.28,99,73,16.56,15.50,1572426057


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

locations = city_data[["Lat", "Lng"]].astype(float)

city_data = city_data.dropna()
humidity = city_data["Humidity"].astype(float)

In [9]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
ideal_city_df = city_data.loc[(city_data["Max Temp"] < 78) & (city_data["Max Temp"] > 68) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)].dropna()
ideal_city_df

,City_ID,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Date
118,118,Port Alfred,ZA,-33.59,26.89,0,46,70.00,8.01,1572426076
134,134,Faya,SA,18.39,42.45,0,13,77.00,4.70,1572426077
158,158,Santa Cruz,BR,-6.22,-36.02,0,79,69.40,9.48,1572426081
191,191,Nouakchott,MR,18.08,-15.98,0,31,75.20,6.93,1572426086
214,214,Jardim,BR,-21.48,-56.15,0,56,77.72,5.57,1572426088
218,218,Road Town,VG,18.42,-64.62,0,92,77.00,5.99,1572426037
223,223,Lebu,ET,8.96,38.73,0,39,69.10,3.91,1572426089
225,225,Puerto Ayacucho,CO,5.66,-67.63,0,97,71.15,2.24,1572426089
321,321,Todos Santos,MX,23.45,-110.23,0,73,73.00,3.00,1572426102
348,348,Zhicheng,CN,35.04,112.59,0,22,72.19,7.11,1572426107


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
118,Port Alfred,ZA,-33.59,26.89,
134,Faya,SA,18.39,42.45,
158,Santa Cruz,BR,-6.22,-36.02,
191,Nouakchott,MR,18.08,-15.98,
214,Jardim,BR,-21.48,-56.15,
218,Road Town,VG,18.42,-64.62,
223,Lebu,ET,8.96,38.73,
225,Puerto Ayacucho,CO,5.66,-67.63,
321,Todos Santos,MX,23.45,-110.23,
348,Zhicheng,CN,35.04,112.59,


In [23]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Lat,Lng,Hotel Name
118,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
134,Faya,SA,18.39,42.45,مقبرة ال مجمل
158,Santa Cruz,BR,-6.22,-36.02,Pousada Trairi
191,Nouakchott,MR,18.08,-15.98,Hôtel Iman
214,Jardim,BR,-21.48,-56.15,Hotel Estância
218,Road Town,VG,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"
223,Lebu,ET,8.96,38.73,Dah-Lak Guest House
225,Puerto Ayacucho,CO,5.66,-67.63,Posada Turistica Manapiare
321,Todos Santos,MX,23.45,-110.23,Hotel California Todos Santos
348,Zhicheng,CN,35.04,112.59,Oriental Jianguo Hotel


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))